# Wrangling Exercises

In [1]:
# regular imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import env

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

# Wrangling
from sklearn.model_selection import train_test_split

## Acquire and Summarize

### 1. Acquire data from the cloud database.

In [2]:
# from our acquire.py:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
sql_query = '''
SELECT 
    *
    FROM properties_2017
    RIGHT JOIN predictions_2017 ON properties_2017.parcelid = predictions_2017.parcelid
    LEFT JOIN propertylandusetype USING (propertylandusetypeid)
    LEFT JOIN airconditioningtype USING (airconditioningtypeid)
    LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
    LEFT JOIN buildingclasstype USING (buildingclasstypeid)
    LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
    LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
    WHERE propertylandusedesc = "Single Family Residential"
    AND predictions_2017.transactiondate LIKE "2017%%"
    AND latitude IS NOT NULL
    AND longitude IS NOT NULL;
'''
    
def get_zillow_data():
    df = pd.read_sql(sql_query, get_connection('zillow'))
    return df

In [3]:
df = get_zillow_data()

In [4]:
df.shape

(52441, 69)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        76 non-null     float64
 1   heatingorsystemtypeid         33935 non-null  float64
 2   buildingclasstypeid           0 non-null      object 
 3   architecturalstyletypeid      70 non-null     float64
 4   airconditioningtypeid         13638 non-null  float64
 5   propertylandusetypeid         52441 non-null  float64
 6   id                            52441 non-null  int64  
 7   parcelid                      52441 non-null  int64  
 8   logerror                      52441 non-null  float64
 9   transactiondate               52441 non-null  object 
 10  id                            52441 non-null  int64  
 11  parcelid                      52441 non-null  int64  
 12  basementsqft                  47 non-null     float64
 13  b

### 2. Summarize your data

In [6]:
for col in df.columns:
    if col != 'id':
        if col != 'parcelid':
            print(col)
            print(df[col].value_counts())

typeconstructiontypeid
6.00    75
4.00     1
Name: typeconstructiontypeid, dtype: int64
heatingorsystemtypeid
 2.00    20736
 7.00    12565
 6.00      517
20.00       85
13.00       16
 1.00        7
18.00        6
10.00        2
24.00        1
Name: heatingorsystemtypeid, dtype: int64
buildingclasstypeid
Series([], Name: buildingclasstypeid, dtype: int64)
architecturalstyletypeid
 7.00    62
 3.00     3
 2.00     2
21.00     2
 8.00     1
Name: architecturalstyletypeid, dtype: int64
airconditioningtypeid
 1.00    11895
13.00     1568
 5.00      159
11.00       16
Name: airconditioningtypeid, dtype: int64
propertylandusetypeid
261.00    52441
Name: propertylandusetypeid, dtype: int64
logerror
 0.00    5
 0.00    5
 0.00    5
 0.00    5
 0.02    4
        ..
 0.02    1
 0.02    1
-0.03    1
 0.03    1
 0.04    1
Name: logerror, Length: 52180, dtype: int64
transactiondate
2017-06-30    810
2017-04-28    617
2017-05-31    566
2017-07-28    546
2017-08-31    528
             ... 
2017-05-0

  345.72    4
5,871.38    3
2,172.75    3
3,347.58    3
  855.16    3
           ..
3,114.95    1
4,050.56    1
4,996.36    1
  683.35    1
6,317.15    1
Name: taxamount, Length: 50834, dtype: int64
taxdelinquencyflag
Y    2079
Name: taxdelinquencyflag, dtype: int64
taxdelinquencyyear
15.00    1131
14.00     560
13.00     129
12.00      81
11.00      79
10.00      47
 9.00      33
 8.00       8
 7.00       8
 6.00       1
99.00       1
 4.00       1
Name: taxdelinquencyyear, dtype: int64
censustractandblock
60,379,201,162,006.00    32
60,379,203,391,054.00    27
60,590,320,571,003.00    24
60,371,417,002,000.00    24
60,590,320,421,002.00    23
                         ..
60,372,714,001,001.00     1
60,371,954,004,007.00     1
60,379,108,051,066.00     1
60,590,758,081,024.00     1
60,375,034,021,020.00     1
Name: censustractandblock, Length: 31404, dtype: int64
propertylandusedesc
Single Family Residential    52441
Name: propertylandusedesc, dtype: int64
airconditioningdesc
Central  

In [7]:
def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # .value_counts()
    # observation of nulls in the dataframe
    '''
    print('SUMMARY REPORT')
    print('=====================================================\n\n')
    print('Dataframe head: ')
    print(df.head(3))
    print('=====================================================\n\n')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================\n\n')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts(), '\n')
        else:
            print(df[col].value_counts(bins=10, sort=False), '\n')
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')

In [8]:
summarize(df)

SUMMARY REPORT


Dataframe head: 
   typeconstructiontypeid  heatingorsystemtypeid buildingclasstypeid  \
0                     NaN                    NaN                None   
1                     NaN                    NaN                None   
2                     NaN                    NaN                None   

   architecturalstyletypeid  airconditioningtypeid  propertylandusetypeid  id  \
0                       NaN                    NaN                 261.00   0   
1                       NaN                    NaN                 261.00   1   
2                       NaN                    NaN                 261.00   2   

   parcelid             logerror transactiondate  ...            taxamount  \
0  14297519                 0.03      2017-01-01  ...            11,013.72   
1  17052889                 0.06      2017-01-01  ...             5,672.48   
2  14186244                 0.01      2017-01-01  ...             6,488.30   

   taxdelinquencyflag  taxdelinquencyye

AttributeError: 'DataFrame' object has no attribute 'dtype'

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute.

In [9]:
# Counting attribute nulls (via column)
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [10]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
buildingclasstypeid,52441,100.00
buildingclassdesc,52441,100.00
finishedsquarefeet15,52441,100.00
finishedsquarefeet13,52441,100.00
basementsqft,52394,99.91
...,...,...
logerror,0,0.00
fips,0,0.00
latitude,0,0.00
parcelid,0,0.00


## Prepare

In [ ]:
# Potential function for removing properties other than single unit properties
def trim_bad_data_zillow(df):
    # If it's not single unit, it's not a single family home.
    df = df[~(df.unitcnt > 1)]
    # If the lot size is smaller than the finished square feet, it's probably bad data or not a single family home
    df = df[~(df.lotsizesquarefeet < df.calculatedfinishedsquarefeet)]
    # If the finished square feet is less than 500 it is likeley an apartment, or bad data
    df = df[~(df.calculatedfinishedsquarefeet < 500)]
    # If there are no bedrooms, likely a loft or bad data
    df = df[~(df.bedroomcnt < 1)]
    # Drop duplicate parcels
    df = df.drop_duplicates(subset='parcelid')
    return df